In [1]:
%cd G:\

G:\


In [2]:
%cd MachineLearning/QAsystem/

G:\MachineLearning\QAsystem


In [3]:
%pwd

'G:\\MachineLearning\\QAsystem'

In [4]:
%cd squad

G:\MachineLearning\QAsystem\squad


In [5]:
%ls

 Volume in drive G is New Volume
 Volume Serial Number is 26DB-12C9

 Directory of G:\MachineLearning\QAsystem\squad

04/05/2023  03:13 AM    <DIR>          .
04/03/2023  08:58 PM    <DIR>          ..
03/29/2023  01:35 PM                45 .gitignore
04/04/2023  05:26 AM    <DIR>          __pycache__
03/29/2023  01:35 PM            10,164 args.py
04/03/2023  08:59 PM    <DIR>          data
04/05/2023  02:36 AM        34,407,442 dev.csv
04/05/2023  02:36 AM        17,318,343 dev_untokenize.csv
04/05/2023  03:19 AM         5,510,814 dev10k.npz
04/05/2023  03:19 AM         5,510,814 dev20k.npz
04/05/2023  03:19 AM         5,510,814 dev30k.npz
04/05/2023  03:19 AM         5,510,814 dev40k.npz
03/29/2023  01:35 PM               254 environment.yml
03/29/2023  01:35 PM             9,190 layers.py
03/29/2023  01:35 PM             1,121 LICENSE
03/29/2023  01:35 PM             2,918 models.py
03/29/2023  01:35 PM             1,380 README.md
04/03/2023  08:59 PM    <DIR>          save
03/29/202

In [6]:
import pandas as pd
import torch
import numpy as np
from tqdm import tqdm
import spacy
import json
from collections import Counter
import csv

G:\anaconda3\envs\MachineLeanring\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def convert_idx(text, tokens):
    current = 0
    spans = []
    for token in tokens:
        current = text.find(token, current)
        if current < 0:
            print(f"Token {token} cannot be found")
            raise Exception()
        spans.append((current, current + len(token)))
        current += len(token)
    return spans

In [8]:
nlp = spacy.blank("en")
def word_tokenize(sent):
    doc = nlp(sent)
    return [token.text for token in doc]

In [34]:
def process(inn,data_type,word_counter, char_counter):

    # writer = csv.writer(csvfile)
    # writer.writerow(["context","question","spans","answers","uuid"])
    print(f"Pre-processing {data_type} examples...")
    examples = []
    eval_examples = {}
    total = 0
    with open(inn,"r") as fh:
        source = json.load(fh)
        for article in tqdm(source["data"]):
                for para in article["paragraphs"]:
                    context = para["context"].replace(
                        "''", '" ').replace("``", '" ')
                    context_tokens = word_tokenize(context)
                    context_chars = [list(token) for token in context_tokens]
                    spans = convert_idx(context, context_tokens)
                    for token in context_tokens:
                        word_counter[token] += len(para["qas"])
                        for char in token:
                            char_counter[char] += len(para["qas"])
                    for qa in para["qas"]:
                        total += 1
                        ques = qa["question"].replace(
                            "''", '" ').replace("``", '" ')
                        ques_tokens = word_tokenize(ques)
                        ques_chars = [list(token) for token in ques_tokens]
                        for token in ques_tokens:
                            word_counter[token] += 1
                            for char in token:
                                char_counter[char] += 1
                        y1s, y2s = [], []
                        answer_texts = []
                        for answer in qa["answers"]:
                            answer_text = answer["text"]
                            answer_start = answer['answer_start']
                            answer_end = answer_start + len(answer_text)
                            answer_texts.append(answer_text)
                            answer_span = []
                            for idx, span in enumerate(spans):
                                if not (answer_end <= span[0] or answer_start >= span[1]):
                                    answer_span.append(idx)
                            y1, y2 = answer_span[0], answer_span[-1]
                            y1s.append(y1)
                            y2s.append(y2)
                        example = {"context_tokens": context_tokens,
                                   "context_chars": context_chars,
                                   "ques_tokens": ques_tokens,
                                   "ques_chars": ques_chars,
                                   "y1s": y1s,
                                   "y2s": y2s,
                                   "id": total}
                        examples.append(example)
                        eval_examples[str(total)] = {"context": context,
                                                     "question": ques,
                                                     "spans": spans,
                                                     "answers": answer_texts,
                                                     "uuid": qa["id"]}
    return examples,eval_examples
    

In [27]:
test, untokenize_test = process('./data/test-v2.0.json')

100%|██████████| 20/20 [00:06<00:00,  3.23it/s]


In [28]:
dev, untokenize_dev = process('./data/dev-v2.0.json')

100%|██████████| 16/16 [00:01<00:00, 10.48it/s]


In [29]:
train,untokenize_train = process('./data/train-v2.0.json')

100%|██████████| 442/442 [00:40<00:00, 10.87it/s]


In [30]:
dev1 = pd.DataFrame(dev)
untokenize_dev1 = pd.DataFrame(untokenize_dev)

test1 = pd.DataFrame(test)
untokenize_test1 = pd.DataFrame(untokenize_test)

train1 = pd.DataFrame(train)
untokenize_train1 = pd.DataFrame(untokenize_train)

In [142]:
dev.to_csv('dev.csv',index=False)
untokenize_dev.to_csv('dev_untokenize.csv',index=False)

test.to_csv('test.csv',index=False)
untokenize_test.to_csv('test_untokenize.csv',index=False)

train.to_csv('train.csv',index=False)
untokenize_train.to_csv('train_untokenize.csv',index=False)

KeyboardInterrupt: 

In [187]:
a = dev.sample(frac=1)
a

,context_tokens,context_chars,ques_tokens,ques_chars,y1s,y2s,id
5850,"[A, simple, case, of, dynamic, equilibrium, oc...","[[A], [s, i, m, p, l, e], [c, a, s, e], [o, f]...","[What, has, to, accounted, for, that, causes, ...","[[W, h, a, t], [h, a, s], [t, o], [a, c, c, o,...","[88, 53, 88, 88]","[89, 60, 89, 89]",5851
3543,"[The, success, of, any, pathogen, depends, on,...","[[T, h, e], [s, u, c, c, e, s, s], [o, f], [a,...","[In, a, type, III, secretion, system, ,, prote...","[[I, n], [a], [t, y, p, e], [I, I, I], [s, e, ...","[105, 105, 105]","[108, 108, 109]",3544
5713,"[With, modern, insights, into, quantum, mechan...","[[W, i, t, h], [m, o, d, e, r, n], [i, n, s, i...","[What, is, the, strongest, main, interaction, ?]","[[W, h, a, t], [i, s], [t, h, e], [s, t, r, o,...","[73, 73, 73, 73, 73, 73]","[73, 73, 74, 73, 73, 75]",5714
5363,"[The, French, population, numbered, about, 75,...","[[T, h, e], [F, r, e, n, c, h], [p, o, p, u, l...","[Where, did, French, fur, trappers, travel, ?]","[[W, h, e, r, e], [d, i, d], [F, r, e, n, c, h...","[88, 88, 86, 88, 87]","[104, 92, 92, 91, 92]",5364
3268,"[Disorders, of, the, immune, system, can, resu...","[[D, i, s, o, r, d, e, r, s], [o, f], [t, h, e...","[What, kind, of, medicine, can, cause, autoimm...","[[W, h, a, t], [k, i, n, d], [o, f], [m, e, d,...",[],[],3269
...,...,...,...,...,...,...,...
3842,"[The, Presiding, Officer, (, or, Deputy, Presi...","[[T, h, e], [P, r, e, s, i, d, i, n, g], [O, f...","[What, members, typically, open, debates, ?]","[[W, h, a, t], [m, e, m, b, e, r, s], [t, y, p...","[52, 52, 52]","[55, 55, 55]",3843
444,"[For, the, complexity, classes, defined, in, t...","[[F, o, r], [t, h, e], [c, o, m, p, l, e, x, i...","[In, what, expression, can, one, expect, to, f...","[[I, n], [w, h, a, t], [e, x, p, r, e, s, s, i...","[41, 41, 41]","[42, 42, 42]",445
5812,"[Historically, ,, forces, were, first, quantit...","[[H, i, s, t, o, r, i, c, a, l, l, y], [,], [f...","[What, must, be, specified, in, order, to, acc...","[[W, h, a, t], [m, u, s, t], [b, e], [s, p, e,...",[],[],5813
6037,"[where, , is, the, mass, of, the, object, ,, ...","[[w, h, e, r, e], [ ], [i, s], [t, h, e], [m, ...","[What, is, another, word, for, centripetal, fo...","[[W, h, a, t], [i, s], [a, n, o, t, h, e, r], ...","[160, 160, 160, 160]","[160, 164, 160, 160]",6038


In [213]:
context_tokens = np.array(a['context_tokens'])
context_chars = np.array(a['context_chars'])
ques_tokens = np.array(a['ques_tokens'])
ques_chars = np.array(a['ques_chars'])
y1s = np.array(a['y1s'])
y2s = np.array(a['y2s'])
id = np.array(a['id'])

np.savez('dev10k.npz',context_tokens=context_tokens[:10000],context_chars=context_chars[:10000],ques_tokens=ques_tokens[:10000],ques_chars=ques_chars[:10000],y1s=y1s[:10000],y2s=y2s[:10000],id=id[:10000])
np.savez('dev20k.npz',context_tokens=context_tokens[:20000],context_chars=context_chars[:20000],ques_tokens=ques_tokens[:20000],ques_chars=ques_chars[:20000],y1s=y1s[:20000],y2s=y2s[:20000],id=id[:20000])
np.savez('dev30k.npz',context_tokens=context_tokens[:30000],context_chars=context_chars[:30000],ques_tokens=ques_tokens[:30000],ques_chars=ques_chars[:30000],y1s=y1s[:30000],y2s=y2s[:30000],id=id[:30000])
np.savez('dev40k.npz',context_tokens=context_tokens[:40000],context_chars=context_chars[:40000],ques_tokens=ques_tokens[:40000],ques_chars=ques_chars[:40000],y1s=y1s[:40000],y2s=y2s[:40000],id=id[:40000])


In [202]:
b = train.sample(frac=1)
b

,context_tokens,context_chars,ques_tokens,ques_chars,y1s,y2s,id
64308,"[Ann, Arbor, was, founded, in, 1824, by, land,...","[[A, n, n], [A, r, b, o, r], [w, a, s], [f, o,...","[What, were, the, names, of, the, founders, wi...","[[W, h, a, t], [w, e, r, e], [t, h, e], [n, a,...",[59],[59],64309
48735,"[In, Alberta, ,, five, bitumen, upgraders, pro...","[[I, n], [A, l, b, e, r, t, a], [,], [f, i, v,...","[Where, is, the, Shell, Oil, upgrader, located...","[[W, h, e, r, e], [i, s], [t, h, e], [S, h, e,...",[],[],48736
100067,"[The, concept, of, ', education, through, recr...","[[T, h, e], [c, o, n, c, e, p, t], [o, f], [']...","[Who, said, , "", A, master, in, the, art, of,...","[[W, h, o], [s, a, i, d], [ ], [""], [A], [m, a...",[],[],100068
123833,"[From, the, early, 16th, century, ,, both, Wes...","[[F, r, o, m], [t, h, e], [e, a, r, l, y], [1,...","[Why, did, Western, and, Eastern, Armenia, com...","[[W, h, y], [d, i, d], [W, e, s, t, e, r, n], ...",[],[],123834
69681,"[In, modern, Iran, ,, he, is, considered, a, n...","[[I, n], [m, o, d, e, r, n], [I, r, a, n], [,]...","[In, what, European, hospital, does, Avicenna,...","[[I, n], [w, h, a, t], [E, u, r, o, p, e, a, n...",[],[],69682
...,...,...,...,...,...,...,...
71204,"[Since, the, beginning, of, the, study, of, el...","[[S, i, n, c, e], [t, h, e], [b, e, g, i, n, n...","[For, what, use, were, non, conductive, materi...","[[F, o, r], [w, h, a, t], [u, s, e], [w, e, r,...",[38],[40],71205
71806,"[Ibn, Khaldun, can, be, regarded, as, the, ear...","[[I, b, n], [K, h, a, l, d, u, n], [c, a, n], ...","[What, was, Ibn, Khaldun, 's, profession, ?]","[[W, h, a, t], [w, a, s], [I, b, n], [K, h, a,...",[8],[10],71807
77680,"[Mīmāṃsā, gave, rise, to, the, study, of, phil...","[[M, ī, m, ā, ṃ, s, ā], [g, a, v, e], [r, i, s...","[What, parts, of, the, Vedas, did, the, Mimams...","[[W, h, a, t], [p, a, r, t, s], [o, f], [t, h,...",[145],[146],77681
33971,"[In, 1907, ,, the, newly, established, Board, ...","[[I, n], [1, 9, 0, 7], [,], [t, h, e], [n, e, ...","[Who, granted, the, Royal, Charted, for, the, ...","[[W, h, o], [g, r, a, n, t, e, d], [t, h, e], ...",[],[],33972


In [211]:
context_tokens = np.array(b['context_tokens'])
context_chars = np.array(b['context_chars'])
ques_tokens = np.array(b['ques_tokens'])
ques_chars = np.array(b['ques_chars'])
y1s = np.array(b['y1s'])
y2s = np.array(b['y2s'])
id = np.array(b['id'])

np.savez('train10k.npz',context_tokens=context_tokens[:10000],context_chars=context_chars[:10000],ques_tokens=ques_tokens[:10000],ques_chars=ques_chars[:10000],y1s=y1s[:10000],y2s=y2s[:10000],id=id[:10000])
np.savez('train20k.npz',context_tokens=context_tokens[:20000],context_chars=context_chars[:20000],ques_tokens=ques_tokens[:20000],ques_chars=ques_chars[:20000],y1s=y1s[:20000],y2s=y2s[:20000],id=id[:20000])
np.savez('train30k.npz',context_tokens=context_tokens[:30000],context_chars=context_chars[:30000],ques_tokens=ques_tokens[:30000],ques_chars=ques_chars[:30000],y1s=y1s[:30000],y2s=y2s[:30000],id=id[:30000])
np.savez('train40k.npz',context_tokens=context_tokens[:40000],context_chars=context_chars[:40000],ques_tokens=ques_tokens[:40000],ques_chars=ques_chars[:40000],y1s=y1s[:40000],y2s=y2s[:40000],id=id[:40000])


In [221]:
import numpy as np

In [222]:
with np.load('train10.npz') as data:
    print(data['context_tokens'])

TypeError: __main__.<lambda>() got multiple values for keyword argument 'allow_pickle'

In [225]:
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, **k)

In [226]:
with np.load('train10.npz') as data:
    print(data['context_tokens'])

RecursionError: maximum recursion depth exceeded

In [15]:
!python train.py -hq

usage: Train a model on SQuAD [-h] [--train_record_file TRAIN_RECORD_FILE]
                              [--dev_record_file DEV_RECORD_FILE]
                              [--test_record_file TEST_RECORD_FILE]
                              [--word_emb_file WORD_EMB_FILE]
                              [--char_emb_file CHAR_EMB_FILE]
                              [--train_eval_file TRAIN_EVAL_FILE]
                              [--dev_eval_file DEV_EVAL_FILE]
                              [--test_eval_file TEST_EVAL_FILE] --name NAME
                              [--max_ans_len MAX_ANS_LEN]
                              [--num_workers NUM_WORKERS]
                              [--save_dir SAVE_DIR] [--batch_size BATCH_SIZE]
                              [--use_squad_v2 USE_SQUAD_V2]
                              [--hidden_size HIDDEN_SIZE]
                              [--num_visuals NUM_VISUALS]
                              [--load_path LOAD_PATH]
                              [--eval_s

In [19]:
untokenize_test

,1,2,3,4,5,6,7,8,9,10,...,5906,5907,5908,5909,5910,5911,5912,5913,5914,5915
context,A prime number (or a prime) is a natural numbe...,A prime number (or a prime) is a natural numbe...,A prime number (or a prime) is a natural numbe...,A prime number (or a prime) is a natural numbe...,A prime number (or a prime) is a natural numbe...,A prime number (or a prime) is a natural numbe...,A prime number (or a prime) is a natural numbe...,A prime number (or a prime) is a natural numbe...,A prime number (or a prime) is a natural numbe...,A prime number (or a prime) is a natural numbe...,...,Alaska Natives are indigenous peoples of Alask...,Alaska Natives are indigenous peoples of Alask...,Alaska Natives are indigenous peoples of Alask...,Alaska Natives are indigenous peoples of Alask...,Alaska Natives are indigenous peoples of Alask...,Alaska Natives are indigenous peoples of Alask...,Alaska Natives are indigenous peoples of Alask...,Alaska Natives are indigenous peoples of Alask...,Alaska Natives are indigenous peoples of Alask...,Alaska Natives are indigenous peoples of Alask...
question,What is the only divisor besides 1 that a prim...,What are numbers greater than 1 that can be di...,What theorem defines the main role of primes i...,Any number larger than 1 can be represented as...,Why must one be excluded in order to preserve ...,What is the only divisor besides 1 that a prod...,What are numbers greater than 1 that can be di...,What can any number larger than 6 can be repre...,A number larger than -3 can be represented as ...,Why must -1 be excluded in order to preserve t...,...,How are Alaskan Natives defined?,How many Alaska Native Regional Corporations a...,When did Ancestors of Alaska Natives migrate i...,Where did the descendants of the third wave of...,Alaska Natives comprise over what percentage o...,Why didnt the Alaskan Natives migrate to the S...,How many language groups do the Natives have?,How many Alaska Natives are enrolled in federa...,Alaska is part of which country?,Where did the descendants of the third wave of...
spans,"[(0, 1), (2, 7), (8, 14), (15, 16), (16, 18), ...","[(0, 1), (2, 7), (8, 14), (15, 16), (16, 18), ...","[(0, 1), (2, 7), (8, 14), (15, 16), (16, 18), ...","[(0, 1), (2, 7), (8, 14), (15, 16), (16, 18), ...","[(0, 1), (2, 7), (8, 14), (15, 16), (16, 18), ...","[(0, 1), (2, 7), (8, 14), (15, 16), (16, 18), ...","[(0, 1), (2, 7), (8, 14), (15, 16), (16, 18), ...","[(0, 1), (2, 7), (8, 14), (15, 16), (16, 18), ...","[(0, 1), (2, 7), (8, 14), (15, 16), (16, 18), ...","[(0, 1), (2, 7), (8, 14), (15, 16), (16, 18), ...",...,"[(0, 6), (7, 14), (15, 18), (19, 29), (30, 37)...","[(0, 6), (7, 14), (15, 18), (19, 29), (30, 37)...","[(0, 6), (7, 14), (15, 18), (19, 29), (30, 37)...","[(0, 6), (7, 14), (15, 18), (19, 29), (30, 37)...","[(0, 6), (7, 14), (15, 18), (19, 29), (30, 37)...","[(0, 6), (7, 14), (15, 18), (19, 29), (30, 37)...","[(0, 6), (7, 14), (15, 18), (19, 29), (30, 37)...","[(0, 6), (7, 14), (15, 18), (19, 29), (30, 37)...","[(0, 6), (7, 14), (15, 18), (19, 29), (30, 37)...","[(0, 6), (7, 14), (15, 18), (19, 29), (30, 37)..."
answers,[],[],[],[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],[]
uuid,84172fcbff821b3efe4c1d9eb,cac4b7f786f7a2cfda50e3b82,3b5bd07c3f7c33aaf32249d06,d8cb92a78d5231a3ecead9f8d,d2854c0eaa7cbfbdba6be22d1,c9fa1ea9b93143aefcd6c45a0,3451b734d34aeae3cbffcd87e,ff1ed5ace0342fdb8e58255f4,dd0e6e3717ffb6a6be7fe7ffe,d5a77fa6639f62de8cec0779d,...,cccd3fec3e9b6f7a29feca9bc,d5bcaf6dedadb6008bed45a95,ff97ce04e7cccded8542b186b,846dbfc429f9e667fcfc5dbdf,cc7acf690ad1cacfe042abc7c,b98cbfe8a1264dbafca6103bc,5bdbb4bc9d411a02ce5567fad,9147fdb4d1ef0570bff794d3f,8e6b2c2cc7194faafb1da5b5a,e95cf81c19f8ac430fdd35cac


In [22]:
def get_embedding(counter, data_type, limit=-1, emb_file=None, vec_size=None, num_vectors=None):
    print(f"Pre-processing {data_type} vectors...")
    embedding_dict = {}
    filtered_elements = [k for k, v in counter.items() if v > limit]
    if emb_file is not None:
        assert vec_size is not None
        with open(emb_file, "r", encoding="utf-8") as fh:
            for line in tqdm(fh, total=num_vectors):
                array = line.split()
                word = "".join(array[0:-vec_size])
                vector = list(map(float, array[-vec_size:]))
                if word in counter and counter[word] > limit:
                    embedding_dict[word] = vector
        print(f"{len(embedding_dict)} / {len(filtered_elements)} tokens have corresponding {data_type} embedding vector")
    else:
        assert vec_size is not None
        for token in filtered_elements:
            embedding_dict[token] = [np.random.normal(
                scale=0.1) for _ in range(vec_size)]
        print(f"{len(filtered_elements)} tokens have corresponding {data_type} embedding vector")

    NULL = "--NULL--"
    OOV = "--OOV--"
    token2idx_dict = {token: idx for idx, token in enumerate(embedding_dict.keys(), 2)}
    token2idx_dict[NULL] = 0
    token2idx_dict[OOV] = 1
    embedding_dict[NULL] = [0. for _ in range(vec_size)]
    embedding_dict[OOV] = [0. for _ in range(vec_size)]
    idx2emb_dict = {idx: embedding_dict[token]
                    for token, idx in token2idx_dict.items()}
    emb_mat = [idx2emb_dict[idx] for idx in range(len(idx2emb_dict))]
    return emb_mat, token2idx_dict

In [20]:
def convert_to_features(args, data, word2idx_dict, char2idx_dict, is_test):
    example = {}
    context, question = data
    context = context.replace("''", '" ').replace("``", '" ')
    question = question.replace("''", '" ').replace("``", '" ')
    example['context_tokens'] = word_tokenize(context)
    example['ques_tokens'] = word_tokenize(question)
    example['context_chars'] = [list(token) for token in example['context_tokens']]
    example['ques_chars'] = [list(token) for token in example['ques_tokens']]

    para_limit = args.test_para_limit if is_test else args.para_limit
    ques_limit = args.test_ques_limit if is_test else args.ques_limit
    char_limit = args.char_limit

    def filter_func(example):
        return len(example["context_tokens"]) > para_limit or \
               len(example["ques_tokens"]) > ques_limit

    if filter_func(example):
        raise ValueError("Context/Questions lengths are over the limit")

    context_idxs = np.zeros([para_limit], dtype=np.int32)
    context_char_idxs = np.zeros([para_limit, char_limit], dtype=np.int32)
    ques_idxs = np.zeros([ques_limit], dtype=np.int32)
    ques_char_idxs = np.zeros([ques_limit, char_limit], dtype=np.int32)

    def _get_word(word):
        for each in (word, word.lower(), word.capitalize(), word.upper()):
            if each in word2idx_dict:
                return word2idx_dict[each]
        return 1

    def _get_char(char):
        if char in char2idx_dict:
            return char2idx_dict[char]
        return 1

    for i, token in enumerate(example["context_tokens"]):
        context_idxs[i] = _get_word(token)

    for i, token in enumerate(example["ques_tokens"]):
        ques_idxs[i] = _get_word(token)

    for i, token in enumerate(example["context_chars"]):
        for j, char in enumerate(token):
            if j == char_limit:
                break
            context_char_idxs[i, j] = _get_char(char)

    for i, token in enumerate(example["ques_chars"]):
        for j, char in enumerate(token):
            if j == char_limit:
                break
            ques_char_idxs[i, j] = _get_char(char)

    return context_idxs, context_char_idxs, ques_idxs, ques_char_idxs

In [64]:
def is_answerable(example):
    return len(example['y2s']) > 0 and len(example['y1s']) > 0

In [65]:
def build_features(examples, data_type, out_file, word2idx_dict, char2idx_dict, is_test=False):
    para_limit = 1000 if is_test else 400
    ques_limit = 100 if is_test else 50
    ans_limit = 30
    char_limit = 16

    def drop_example(ex, is_test_=False):
        if is_test_:
            drop = False
        else:
            drop = len(ex["context_tokens"]) > para_limit or \
                   len(ex["ques_tokens"]) > ques_limit or \
                   (is_answerable(ex) and
                    ex["y2s"][0] - ex["y1s"][0] > ans_limit)

        return drop

    print(f"Converting {data_type} examples to indices...")
    total = 0
    total_ = 0
    meta = {}
    context_idxs = []
    context_char_idxs = []
    ques_idxs = []
    ques_char_idxs = []
    y1s = []
    y2s = []
    ids = []
    for n, example in tqdm(enumerate(examples)):
        total_ += 1

        if drop_example(example, is_test):
            continue

        total += 1

        def _get_word(word):
            for each in (word, word.lower(), word.capitalize(), word.upper()):
                if each in word2idx_dict:
                    return word2idx_dict[each]
            return 1

        def _get_char(char):
            if char in char2idx_dict:
                return char2idx_dict[char]
            return 1

        context_idx = np.zeros([para_limit], dtype=np.int32)
        context_char_idx = np.zeros([para_limit, char_limit], dtype=np.int32)
        ques_idx = np.zeros([ques_limit], dtype=np.int32)
        ques_char_idx = np.zeros([ques_limit, char_limit], dtype=np.int32)

        for i, token in enumerate(example["context_tokens"]):
            context_idx[i] = _get_word(token)
        context_idxs.append(context_idx)

        for i, token in enumerate(example["ques_tokens"]):
            ques_idx[i] = _get_word(token)
        ques_idxs.append(ques_idx)

        for i, token in enumerate(example["context_chars"]):
            for j, char in enumerate(token):
                if j == char_limit:
                    break
                context_char_idx[i, j] = _get_char(char)
        context_char_idxs.append(context_char_idx)

        for i, token in enumerate(example["ques_chars"]):
            for j, char in enumerate(token):
                if j == char_limit:
                    break
                ques_char_idx[i, j] = _get_char(char)
        ques_char_idxs.append(ques_char_idx)

        if is_answerable(example):
            start, end = example["y1s"][-1], example["y2s"][-1]
        else:
            start, end = -1, -1

        y1s.append(start)
        y2s.append(end)
        ids.append(example["id"])

    np.savez(out_file,
             context_idxs=np.array(context_idxs),
             context_char_idxs=np.array(context_char_idxs),
             ques_idxs=np.array(ques_idxs),
             ques_char_idxs=np.array(ques_char_idxs),
             y1s=np.array(y1s),
             y2s=np.array(y2s),
             ids=np.array(ids))
    print(f"Built {total} / {total_} instances of features in total")
    meta["total"] = total
    return meta

In [35]:
word_counter, char_counter = Counter(), Counter()
train_examples, train_eval = process('./data/train-v2.0.json',"train", word_counter, char_counter)
word_emb_mat, word2idx_dict = get_embedding(word_counter, 'word', emb_file='./data/glove.840B.300d/glove.840B.300d.txt', vec_size=300, num_vectors=2196017)
char_emb_mat, char2idx_dict = get_embedding(char_counter, 'char', emb_file=None, vec_size=64)

dev_examples, dev_eval = process('./data/dev-v2.0.json',"dev", word_counter, char_counter)


Pre-processing train examples...


100%|██████████| 442/442 [00:33<00:00, 13.30it/s]


Pre-processing word vectors...


100%|██████████| 2196017/2196017 [06:18<00:00, 5807.86it/s]


88427 / 106876 tokens have corresponding word embedding vector
Pre-processing char vectors...
1374 tokens have corresponding char embedding vector
Pre-processing dev examples...


100%|██████████| 16/16 [00:00<00:00, 16.99it/s]


In [43]:
a = np.array(train_examples)

In [46]:
b = np.random.choice(a,10000)

In [49]:
def random_choice_K (K,examples):
    a = np.array(examples)
    b = np.random.choice(a,K)
    return b

In [51]:
c  = random_choice_K(10000,train_examples)
c1 = random_choice_K(20000,train_examples)
c2 = random_choice_K(30000,train_examples)
c3 = random_choice_K(40000,train_examples)

array([{'context_tokens': ['The', 'British', 'declaration', 'of', 'war', 'on', 'Germany', 'and', 'its', 'allies', 'also', 'committed', 'the', 'colonies', 'and', 'Dominions', ',', 'which', 'provided', 'invaluable', 'military', ',', 'financial', 'and', 'material', 'support', '.', 'Over', '2.5', 'million', 'men', 'served', 'in', 'the', 'armies', 'of', 'the', 'Dominions', ',', 'as', 'well', 'as', 'many', 'thousands', 'of', 'volunteers', 'from', 'the', 'Crown', 'colonies', '.', 'The', 'contributions', 'of', 'Australian', 'and', 'New', 'Zealand', 'troops', 'during', 'the', '1915', 'Gallipoli', 'Campaign', 'against', 'the', 'Ottoman', 'Empire', 'had', 'a', 'great', 'impact', 'on', 'the', 'national', 'consciousness', 'at', 'home', ',', 'and', 'marked', 'a', 'watershed', 'in', 'the', 'transition', 'of', 'Australia', 'and', 'New', 'Zealand', 'from', 'colonies', 'to', 'nations', 'in', 'their', 'own', 'right', '.', 'The', 'countries', 'continue', 'to', 'commemorate', 'this', 'occasion', 'on', 'Anz

In [57]:
 test_examples, test_eval = process('./data/test-v2.0.json', "test", word_counter, char_counter)

Pre-processing test examples...


100%|██████████| 20/20 [00:00<00:00, 30.73it/s]


In [60]:
char2idx_dict


{'B': 2,
 'e': 3,
 'y': 4,
 'o': 5,
 'n': 6,
 'c': 7,
 'é': 8,
 'G': 9,
 'i': 10,
 's': 11,
 'l': 12,
 'K': 13,
 'w': 14,
 '-': 15,
 'C': 16,
 'a': 17,
 'r': 18,
 't': 19,
 '(': 20,
 '/': 21,
 'b': 22,
 'ː': 23,
 'ˈ': 24,
 'j': 25,
 'ɒ': 26,
 'ɪ': 27,
 'Y': 28,
 'O': 29,
 'N': 30,
 ')': 31,
 'S': 32,
 'p': 33,
 'm': 34,
 '4': 35,
 ',': 36,
 '1': 37,
 '9': 38,
 '8': 39,
 'A': 40,
 'g': 41,
 'd': 42,
 'u': 43,
 '.': 44,
 'H': 45,
 'T': 46,
 'x': 47,
 'h': 48,
 'f': 49,
 'v': 50,
 '0': 51,
 'R': 52,
 '&': 53,
 'D': 54,
 "'": 55,
 'M': 56,
 'L': 57,
 '2': 58,
 '3': 59,
 '"': 60,
 'z': 61,
 'W': 62,
 '?': 63,
 'I': 64,
 ' ': 65,
 'k': 66,
 'F': 67,
 'J': 68,
 '5': 69,
 '6': 70,
 'à': 71,
 'V': 72,
 'P': 73,
 'Z': 74,
 'E': 75,
 ';': 76,
 'q': 77,
 '7': 78,
 'X': 79,
 'U': 80,
 ':': 81,
 '$': 82,
 '[': 83,
 ']': 84,
 '—': 85,
 'Q': 86,
 '#': 87,
 '–': 88,
 '%': 89,
 'è': 90,
 'ç': 91,
 'ʃ': 92,
 'ʊ': 93,
 'æ': 94,
 '\u200b': 95,
 'ʁ': 96,
 'ɑ': 97,
 '̃': 98,
 'ɔ': 99,
 'ɛ': 100,
 'ń': 101,
 

In [66]:
build_features(train_examples, "train",'./train.npz', word2idx_dict, char2idx_dict)
dev_meta = build_features(dev_examples, "dev",'./dev.npz', word2idx_dict, char2idx_dict)

Converting train examples to indices...


130319it [00:48, 2709.41it/s]


Built 129900 / 130319 instances of features in total
Converting dev examples to indices...


6078it [00:02, 2476.45it/s]


Built 5951 / 6078 instances of features in total


In [68]:
train10k = random_choice_K(10000,train_examples)
train20k = random_choice_K(20000,train_examples)
train30k = random_choice_K(30000,train_examples)
train40k = random_choice_K(40000,train_examples)

In [70]:
build_features(train10k, "train",'./train10k.npz', word2idx_dict, char2idx_dict)
build_features(train20k, "train",'./train20k.npz', word2idx_dict, char2idx_dict)
build_features(train30k, "train",'./train30k.npz', word2idx_dict, char2idx_dict)
build_features(train40k, "train",'./train40k.npz', word2idx_dict, char2idx_dict)

Converting train examples to indices...



0it [00:00, ?it/s]
249it [00:00, 2476.77it/s]
515it [00:00, 2576.19it/s]
804it [00:00, 2711.57it/s]
1106it [00:00, 2825.83it/s]
1389it [00:00, 2791.76it/s]
1677it [00:00, 2820.75it/s]
1960it [00:00, 2764.97it/s]
2237it [00:00, 2753.19it/s]
2513it [00:00, 2557.90it/s]
2772it [00:01, 2480.18it/s]
3022it [00:01, 2429.42it/s]
3267it [00:01, 2303.61it/s]
3499it [00:01, 2248.54it/s]
3728it [00:01, 2256.79it/s]
3957it [00:01, 2261.72it/s]
4184it [00:01, 2160.94it/s]
4402it [00:01, 2028.59it/s]
4607it [00:01, 1869.87it/s]
4816it [00:02, 1927.04it/s]
5030it [00:02, 1984.63it/s]
5268it [00:02, 2092.88it/s]
5480it [00:02, 2097.49it/s]
5719it [00:02, 2179.45it/s]
5996it [00:02, 2350.67it/s]
6270it [00:02, 2461.34it/s]
6518it [00:02, 2465.38it/s]
6766it [00:02, 2144.35it/s]
7019it [00:03, 2247.81it/s]
7279it [00:03, 2342.70it/s]
7519it [00:03, 2258.11it/s]
7754it [00:03, 2280.34it/s]
8030it [00:03, 2413.58it/s]
8324it [00:03, 2561.29it/s]
8583it [00:03, 2479.02it/s]
8834it [00:03, 2252.00it/s]
907

Built 9967 / 10000 instances of features in total
Converting train examples to indices...



0it [00:00, ?it/s]
273it [00:00, 2715.76it/s]
545it [00:00, 2617.91it/s]
808it [00:00, 2363.29it/s]
1071it [00:00, 2455.02it/s]
1319it [00:00, 2335.15it/s]
1555it [00:00, 2316.40it/s]
1829it [00:00, 2443.27it/s]
2088it [00:00, 2487.64it/s]
2338it [00:00, 2282.77it/s]
2580it [00:01, 2318.12it/s]
2855it [00:01, 2438.90it/s]
3142it [00:01, 2560.43it/s]
3429it [00:01, 2646.72it/s]
3696it [00:01, 2457.34it/s]
3946it [00:01, 2460.73it/s]
4223it [00:01, 2547.41it/s]
4481it [00:01, 2362.78it/s]
4722it [00:01, 2339.37it/s]
4983it [00:02, 2410.98it/s]
5242it [00:02, 2458.32it/s]
5495it [00:02, 2475.30it/s]
5744it [00:02, 2297.79it/s]
5986it [00:02, 2328.25it/s]
6241it [00:02, 2387.93it/s]
6510it [00:02, 2473.67it/s]
6763it [00:02, 2489.65it/s]
7014it [00:02, 2342.07it/s]
7268it [00:03, 2394.51it/s]
7510it [00:03, 2357.51it/s]
7748it [00:03, 2349.30it/s]
8000it [00:03, 2394.88it/s]
8247it [00:03, 2412.86it/s]
8489it [00:03, 2383.70it/s]
8745it [00:03, 2431.56it/s]
8989it [00:03, 2419.45it/s]
923

Built 19942 / 20000 instances of features in total
Converting train examples to indices...



0it [00:00, ?it/s]
270it [00:00, 2684.93it/s]
539it [00:00, 2663.26it/s]
815it [00:00, 2700.23it/s]
1086it [00:00, 2625.55it/s]
1384it [00:00, 2745.38it/s]
1702it [00:00, 2887.90it/s]
1992it [00:00, 2867.58it/s]
2280it [00:00, 2796.28it/s]
2563it [00:00, 2801.87it/s]
2844it [00:01, 2783.22it/s]
3123it [00:01, 2727.53it/s]
3399it [00:01, 2732.87it/s]
3683it [00:01, 2762.36it/s]
3961it [00:01, 2767.07it/s]
4238it [00:01, 2766.83it/s]
4515it [00:01, 2763.79it/s]
4792it [00:01, 2658.09it/s]
5062it [00:01, 2666.10it/s]
5330it [00:01, 2660.25it/s]
5597it [00:02, 2659.58it/s]
5864it [00:02, 2642.38it/s]
6129it [00:02, 2601.32it/s]
6403it [00:02, 2637.86it/s]
6681it [00:02, 2679.18it/s]
6950it [00:02, 2624.52it/s]
7213it [00:02, 2606.79it/s]
7478it [00:02, 2619.18it/s]
7745it [00:02, 2630.15it/s]
8011it [00:02, 2634.76it/s]
8293it [00:03, 2688.83it/s]
8577it [00:03, 2728.96it/s]
8869it [00:03, 2781.52it/s]
9151it [00:03, 2789.30it/s]
9430it [00:03, 2736.17it/s]
9704it [00:03, 2733.03it/s]
997

Built 29906 / 30000 instances of features in total
Converting train examples to indices...



0it [00:00, ?it/s]
306it [00:00, 3041.05it/s]
611it [00:00, 2573.65it/s]
906it [00:00, 2720.67it/s]
1203it [00:00, 2809.76it/s]
1496it [00:00, 2842.64it/s]
1805it [00:00, 2921.70it/s]
2099it [00:00, 1969.55it/s]
2363it [00:00, 2124.35it/s]
2637it [00:01, 2277.37it/s]
2906it [00:01, 2383.80it/s]
3199it [00:01, 2532.12it/s]
3492it [00:01, 2639.37it/s]
3768it [00:01, 2636.23it/s]
4066it [00:01, 2729.41it/s]
4345it [00:01, 2721.45it/s]
4622it [00:01, 2735.22it/s]
4899it [00:01, 2484.30it/s]
5154it [00:02, 2063.56it/s]
5439it [00:02, 2254.31it/s]
5730it [00:02, 2420.30it/s]
6021it [00:02, 2551.74it/s]
6310it [00:02, 2641.46it/s]
6602it [00:02, 2712.45it/s]
6882it [00:02, 2737.64it/s]
7161it [00:02, 2736.89it/s]
7448it [00:02, 2770.31it/s]
7728it [00:03, 2588.13it/s]
7999it [00:03, 2615.27it/s]
8279it [00:03, 2664.91it/s]
8548it [00:03, 2665.06it/s]
8837it [00:03, 2728.65it/s]
9112it [00:03, 2733.06it/s]
9391it [00:03, 2746.97it/s]
9673it [00:03, 2767.28it/s]
9951it [00:03, 2756.63it/s]
102

Built 39879 / 40000 instances of features in total


{'total': 39879}

In [72]:
dev10k = random_choice_K(10000,dev_examples)
dev20k = random_choice_K(20000,dev_examples)
dev30k = random_choice_K(30000,dev_examples)
dev40k = random_choice_K(40000,dev_examples)

In [73]:
build_features(dev10k, "dev",'./dev10k.npz', word2idx_dict, char2idx_dict)
build_features(dev20k, "dev",'./dev20k.npz', word2idx_dict, char2idx_dict)
build_features(dev30k, "dev",'./dev30k.npz', word2idx_dict, char2idx_dict)
build_features(dev40k, "dev",'./dev40k.npz', word2idx_dict, char2idx_dict)

Converting dev examples to indices...



0it [00:00, ?it/s]
247it [00:00, 2455.86it/s]
493it [00:00, 2369.31it/s]
739it [00:00, 2392.52it/s]
1014it [00:00, 2524.96it/s]
1297it [00:00, 2632.39it/s]
1584it [00:00, 2707.81it/s]
1865it [00:00, 2736.46it/s]
2141it [00:00, 2739.48it/s]
2416it [00:00, 2738.37it/s]
2690it [00:01, 2652.52it/s]
2956it [00:01, 2645.05it/s]
3242it [00:01, 2704.87it/s]
3521it [00:01, 2726.09it/s]
3796it [00:01, 2729.04it/s]
4070it [00:01, 2711.25it/s]
4342it [00:01, 2642.63it/s]
4619it [00:01, 2675.80it/s]
4898it [00:01, 2704.31it/s]
5169it [00:01, 2654.97it/s]
5435it [00:02, 2582.53it/s]
5694it [00:02, 2543.82it/s]
5949it [00:02, 2541.82it/s]
6204it [00:02, 2540.31it/s]
6459it [00:02, 2539.04it/s]
6714it [00:02, 2514.97it/s]
6973it [00:02, 2533.57it/s]
7251it [00:02, 2602.30it/s]
7512it [00:02, 2584.30it/s]
7772it [00:02, 2585.09it/s]
8036it [00:03, 2597.45it/s]
8319it [00:03, 2661.59it/s]
8587it [00:03, 2662.80it/s]
8854it [00:03, 2365.17it/s]
9097it [00:03, 2165.74it/s]
9321it [00:03, 2151.25it/s]
954

Built 9779 / 10000 instances of features in total
Converting dev examples to indices...



0it [00:00, ?it/s]
180it [00:00, 1795.52it/s]
369it [00:00, 1849.58it/s]
561it [00:00, 1877.33it/s]
749it [00:00, 1783.01it/s]
928it [00:00, 1740.32it/s]
1114it [00:00, 1776.25it/s]
1293it [00:00, 1676.64it/s]
1514it [00:00, 1833.22it/s]
1700it [00:00, 1780.01it/s]
1880it [00:01, 1772.67it/s]
2059it [00:01, 1683.74it/s]
2245it [00:01, 1726.28it/s]
2447it [00:01, 1808.07it/s]
2657it [00:01, 1891.15it/s]
2868it [00:01, 1952.06it/s]
3114it [00:01, 2098.55it/s]
3372it [00:01, 2238.88it/s]
3646it [00:01, 2387.61it/s]
3912it [00:01, 2468.09it/s]
4195it [00:02, 2571.86it/s]
4469it [00:02, 2617.14it/s]
4746it [00:02, 2660.17it/s]
5030it [00:02, 2709.59it/s]
5302it [00:02, 2709.77it/s]
5588it [00:02, 2750.01it/s]
5864it [00:02, 2627.04it/s]
6128it [00:02, 2587.74it/s]
6388it [00:02, 2566.37it/s]
6646it [00:03, 2154.89it/s]
6873it [00:03, 1965.73it/s]
7080it [00:03, 1890.19it/s]
7329it [00:03, 2038.90it/s]
7578it [00:03, 2154.85it/s]
7816it [00:03, 2214.48it/s]
8078it [00:03, 2325.00it/s]
8335i

Built 19597 / 20000 instances of features in total
Converting dev examples to indices...



0it [00:00, ?it/s]
224it [00:00, 2226.31it/s]
478it [00:00, 2403.89it/s]
737it [00:00, 2487.43it/s]
1008it [00:00, 2569.20it/s]
1275it [00:00, 2600.35it/s]
1563it [00:00, 2690.51it/s]
1833it [00:00, 2654.28it/s]
2099it [00:00, 2645.62it/s]
2388it [00:00, 2720.71it/s]
2669it [00:01, 2743.42it/s]
2944it [00:01, 2540.43it/s]
3218it [00:01, 2597.23it/s]
3504it [00:01, 2669.38it/s]
3789it [00:01, 2717.16it/s]
4063it [00:01, 2533.35it/s]
4331it [00:01, 2569.92it/s]
4609it [00:01, 2625.76it/s]
4887it [00:01, 2666.38it/s]
5166it [00:01, 2700.40it/s]
5445it [00:02, 2720.37it/s]
5718it [00:02, 2633.47it/s]
5983it [00:02, 2406.80it/s]
6230it [00:02, 2420.77it/s]
6476it [00:02, 2274.72it/s]
6707it [00:02, 2216.43it/s]
6931it [00:02, 2174.63it/s]
7150it [00:02, 1839.50it/s]
7343it [00:03, 1210.83it/s]
7566it [00:03, 1401.90it/s]
7788it [00:03, 1573.76it/s]
7993it [00:03, 1683.64it/s]
8186it [00:03, 1627.01it/s]
8366it [00:03, 1655.17it/s]
8556it [00:03, 1716.66it/s]
8786it [00:03, 1871.59it/s]
905

Built 29356 / 30000 instances of features in total
Converting dev examples to indices...



0it [00:00, ?it/s]
243it [00:00, 2414.90it/s]
523it [00:00, 2632.94it/s]
799it [00:00, 2684.52it/s]
1094it [00:00, 2782.26it/s]
1373it [00:00, 2780.01it/s]
1652it [00:00, 2698.10it/s]
1923it [00:00, 2412.48it/s]
2170it [00:00, 2404.69it/s]
2428it [00:00, 2451.92it/s]
2676it [00:01, 2413.86it/s]
2922it [00:01, 2423.58it/s]
3184it [00:01, 2477.22it/s]
3454it [00:01, 2538.76it/s]
3715it [00:01, 2555.64it/s]
4006it [00:01, 2656.35it/s]
4277it [00:01, 2671.22it/s]
4545it [00:01, 2653.74it/s]
4820it [00:01, 2677.81it/s]
5090it [00:01, 2680.24it/s]
5359it [00:02, 2662.30it/s]
5626it [00:02, 2514.52it/s]
5880it [00:02, 2221.50it/s]
6117it [00:02, 2257.68it/s]
6400it [00:02, 2411.18it/s]
6659it [00:02, 2457.89it/s]
6909it [00:02, 2207.34it/s]
7137it [00:02, 2171.89it/s]
7409it [00:02, 2317.27it/s]
7673it [00:03, 2403.14it/s]
7954it [00:03, 2515.19it/s]
8229it [00:03, 2579.27it/s]
8490it [00:03, 2524.59it/s]
8751it [00:03, 2545.42it/s]
9014it [00:03, 2566.29it/s]
9287it [00:03, 2610.50it/s]
955

Built 39167 / 40000 instances of features in total


{'total': 39167}

In [75]:
len(train_examples)

130319

In [77]:
len(dev_examples)

6078

In [78]:
!python train.py -h

usage: Train a model on SQuAD [-h] [--train_record_file TRAIN_RECORD_FILE]
                              [--dev_record_file DEV_RECORD_FILE]
                              [--test_record_file TEST_RECORD_FILE]
                              [--word_emb_file WORD_EMB_FILE]
                              [--char_emb_file CHAR_EMB_FILE]
                              [--train_eval_file TRAIN_EVAL_FILE]
                              [--dev_eval_file DEV_EVAL_FILE]
                              [--test_eval_file TEST_EVAL_FILE] --name NAME
                              [--max_ans_len MAX_ANS_LEN]
                              [--num_workers NUM_WORKERS]
                              [--save_dir SAVE_DIR] [--batch_size BATCH_SIZE]
                              [--use_squad_v2 USE_SQUAD_V2]
                              [--hidden_size HIDDEN_SIZE]
                              [--num_visuals NUM_VISUALS]
                              [--load_path LOAD_PATH]
                              [--eval_s

In [79]:
!python train.py -n baseline --train_record_file ./train10k.npz

^C
